In [1]:
#Read in email file

with open('./example/email1.txt', 'r') as myfile:
  email = myfile.read()

In [2]:
import pandas as pd
len(email)

2474

In [3]:
print(email)

You are invited to the following event:

URBAN FORESTRY: LESSONS LEARNED, CHALLENGES AND OPPORTUNITIES AHEAD
divider
 	
Event to be held at the following time, date, and location:

Wednesday, November 9, 2016 from 6:00 PM to 7:30 PM (EST)

Cornell University Architecture, Art, Planning
26 Broadway (20th floor) 
New York, NY 10004

View Map

Attend Event	
Share this event:
Facebook	Twitter	LinkedIn divider
Join Us

On November 9th at Cornell University's Architecture Art Planning for a presentation from author Jill Jonnes, and panel discussion with expert voices in American forestry. We will discuss the past, present and future of Uran Forestry with panelists: Fiona Watt, NYC Parks, Todd Forrest, New York Botanical Garden, Joseph Gittelman, ALB Expert and Jill Jonnes. Jill will join the panel after giving her presentation. 

 

Jill Jonnes, author of Eiffel�s Tower, Conquering Gotham, Empires of Light, and South Bronx Rising, will read a selection from her new title, Urban Forests: A Na

In [4]:
#preprocessing text, split string into list of words
escapes = [chr(char) for char in range(1,32)]
escapes.append('\x92')
escapes.append('\x97')

#remove escape chars as spaces
strList = email\
.replace('\n',' ')\
.replace('\r', ' ')\
.replace('\t', ' ')\
.translate(None, ''.join(escapes))\
.lower()

#lowercase and lemmatize, should i lemmatize?


strList


"you are invited to the following event:    urban forestry: lessons learned, challenges and opportunities ahead  divider      event to be held at the following time, date, and location:    wednesday, november 9, 2016 from 6:00 pm to 7:30 pm (est)    cornell university architecture, art, planning  26 broadway (20th floor)   new york, ny 10004    view map    attend event   share this event:  facebook twitter linkedin divider  join us    on november 9th at cornell university's architecture art planning for a presentation from author jill jonnes, and panel discussion with expert voices in american forestry. we will discuss the past, present and future of uran forestry with panelists: fiona watt, nyc parks, todd forrest, new york botanical garden, joseph gittelman, alb expert and jill jonnes. jill will join the panel after giving her presentation.          jill jonnes, author of eiffels tower, conquering gotham, empires of light, and south bronx rising, will read a selection from her new ti

In [5]:
srsFoods = pd.read_csv('./data/Review_table_Reduced.csv')['text']

In [6]:
len(srsFoods)

622446

In [7]:
dfFoods = pd.concat([srsFoods, pd.Series([1]*len(srsFoods), name = 'isFood')], axis = 1)

In [8]:
dfFoods.head()

,text,isFood
0,Not the same Domino's pizza I ate too much of ...,1
1,Easy ordering-on-line. Made exactly as I asked...,1
2,OK OK... as a Proud Italian I hope my momma do...,1
3,never has crushed peppers! \n\nbut the pizza a...,1
4,"The pizza wasn't horrible, but the deal was a...",1


In [ ]:
import os

#Read in data from email files

data = []

for f in os.listdir('./data/enron'):
    with open('./data/enron/' + f, 'r') as myfile:
        data.append(myfile.read())
        
enron = pd.Series(data, name = 'text')

#Only to be used on enron emails
def rmMsgID(strEnron):
    return strEnron[57:]

enron = enron.apply(rmMsgID)
#len('Message-ID: <2262872.1075863600393.JavaMail.evans@thyme>')
dfSpam = pd.concat([enron, pd.Series([0]*len(enron), name = 'isFood')], axis = 1)
dfSpam.to_csv('enron.csv')
dfSpam.head()

,text,isFood
0,"\r\nDate: Wed, 13 Dec 2000 18:41:00 -0800 (PST...",0
1,"\r\nDate: Wed, 13 Dec 2000 08:35:00 -0800 (PST...",0
2,"\nDate: Mon, 9 Oct 2000 07:16:00 -0700 (PDT)\r...",0
3,"\r\nDate: Mon, 9 Oct 2000 07:00:00 -0700 (PDT)...",0
4,"\nDate: Thu, 5 Oct 2000 06:26:00 -0700 (PDT)\r...",0


In [11]:
#Union two dataframes
dfData = dfSpam.append(dfFoods)
dfData.head()

,text,isFood
0,"\r\nDate: Wed, 13 Dec 2000 18:41:00 -0800 (PST...",0
1,"\r\nDate: Wed, 13 Dec 2000 08:35:00 -0800 (PST...",0
2,"\nDate: Mon, 9 Oct 2000 07:16:00 -0700 (PDT)\r...",0
3,"\r\nDate: Mon, 9 Oct 2000 07:00:00 -0700 (PDT)...",0
4,"\nDate: Thu, 5 Oct 2000 06:26:00 -0700 (PDT)\r...",0


In [12]:
#preprocessing text, split string into list of words
escapes = [chr(char) for char in range(1,32)]
escapes.append('\x92')
escapes.append('\x97')

import string

#remove escape chars as spaces
def strStrip(text):
    return text\
.replace('\n',' ')\
.replace('\r', ' ')\
.replace('\t', ' ')\
.translate(None, ''.join(escapes))\
.translate(None, string.punctuation)\
.lower()

def strUni(text):
    return unicode(text, 'latin-1')

dfData['text'] = dfData['text'].apply(strStrip)
dfData['text'] = dfData['text'].apply(strUni)


In [13]:
#Split into train and test set
import numpy as np
from sklearn.model_selection import train_test_split

X = dfData['text']
y = dfData['isFood']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(645916, 929505)

In [15]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(645916, 929505)

In [16]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [17]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

['text_clf.pkl']

In [19]:
text_clf.fit(X_train, y_train)

from sklearn.externals import joblib
joblib.dump(text_clf, 'text_clf.pkl')

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.99901929351413532

In [21]:
#99% Accuracy, now to test it on our other email files
#Read in data from email files

freeFood = []

for f in os.listdir('./example'):
    with open('./example/' + f, 'r') as myfile:
        freeFood.append(myfile.read())
        
srsTest = pd.Series(freeFood, name = 'text')

dfTest = pd.concat([srsTest, pd.Series([1]*len(srsTest), name = 'isFood')], axis = 1)

#preprocess dfTest
#preprocessing text, split string into list of words

dfTest['text'] = dfTest['text'].apply(strStrip)
dfTest['text'] = dfTest['text'].apply(strUni)

predicted = text_clf.predict(dfTest['text'])
#predicted

In [22]:
predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])